In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
import pandas as pd
import rasterio.plot

import overpy
import geopandas as gpd
# import ee
from shapely.geometry import Polygon, box
import os
# import geemap
import json
import multiprocessing

from tqdm import tqdm
import time


In [2]:
# os.chdir('/Users/yer1k/Documents/Master/Duke/2023 Fall/Capstone/solar_wastewater/30_result/')
os.chdir('/Users/yer1k/Library/CloudStorage/GoogleDrive-iamtsubasaozora@gmail.com/My Drive/Capstone Project/Data/')

In [3]:
api = overpy.Overpass()

In [4]:

# Fetch all wastewater treatment plants within California's boundary (# changed to Texas)
query = f"""
    area[admin_level=4]["name"="Georgia"]->.searchArea; 
    (
    way["man_made"="wastewater_plant"](area.searchArea);
    );
    (._;>;);
    out body;
    """

result = api.query(query)

plants = {}

for way in result.ways:
    # Use the name of the plant or its ID if the name is not available
    plant_name = way.tags.get("name", f"Plant_{way.id}")

    # Extract nodes lat and lon without id
    nodes_coords = [(node.lon, node.lat) for node in way.nodes]

    # get rid of "Decimal"  in the coordinates
    nodes_coords = [tuple(map(float, i)) for i in nodes_coords]
    
    plants[plant_name] = nodes_coords

# plants_with_official_name = {key: value for key, value in plants.items() if not key.startswith("Plant_")}


geoms = [Polygon(plants[key]) for key in plants]
df = gpd.GeoDataFrame({'WWTP_name': list(plants.keys()), 'geometry': geoms}, crs="EPSG:4326")
df["centroid"] = df.to_crs('+proj=cea').centroid.to_crs(epsg=4326)



In [5]:
df

,WWTP_name,geometry,centroid
0,Plant_31457419,"POLYGON ((-84.97626 32.41316, -84.97538 32.413...",POINT (-84.97721 32.41063)
1,Plant_34770800,"POLYGON ((-84.56326 33.34549, -84.56406 33.345...",POINT (-84.56446 33.34747)
2,Plant_34770998,"POLYGON ((-84.55196 33.74347, -84.55110 33.743...",POINT (-84.55363 33.74202)
3,South Cobb Water Reclamation Facility,"POLYGON ((-84.53757 33.77225, -84.54008 33.773...",POINT (-84.53795 33.77505)
4,Plant_34771069,"POLYGON ((-83.59370 33.95150, -83.59415 33.950...",POINT (-83.59615 33.95137)
...,...,...,...
223,Plant_1186316344,"POLYGON ((-85.34845 34.46021, -85.34848 34.460...",POINT (-85.34755 34.45935)
224,Former Chatham County Sanitary Sewerage Lift S...,"POLYGON ((-81.05812 31.97596, -81.05809 31.976...",POINT (-81.05815 31.97597)
225,Fowler Water Reclamation Facility,"POLYGON ((-84.21200 34.15300, -84.21215 34.154...",POINT (-84.21485 34.15379)
226,Omega Treatment Plant,"POLYGON ((-83.59347 31.34101, -83.59364 31.340...",POINT (-83.59349 31.34089)


In [6]:
!ls

Archive                         Status Track.gsheet
PPT From Christine              WWTP Lists
Satellite Imagery_Automated RGB tif_bounding_box


In [7]:
# Added progress bar
# use loop to open the image in tif folder, and plot the bounding box on it
# read all the file names in the folder
directory = 'tif_bounding_box/Images of All 50 States/Georgia'
files = os.listdir(directory)

for file in tqdm(files, desc='plotting bounding box'):
    try:
        with rasterio.open(os.path.join(directory, file)) as src:
            # if file.startswith('download'):
                # continue
            # Read the image data
            img = src.read([1, 2, 3])  # Reads the first three bands for RGB
            # file name without extension
            name = os.path.splitext(file)[0]
            # find the corresponding bounding box
            df_name = df.loc[df['WWTP_name'] == name, :]
            # plot the bounding box
            fig, ax = plt.subplots(figsize=(8,8))
            # Plot the raster
            ax = rasterio.plot.show(img, ax=ax, transform=src.transform)
            # Plot the geometry on the ax axes instance
            df_name.boundary.plot(ax=ax, color='red', linewidth=1.5)
            plt.axis('off')
            # save the figure
            plt.savefig(os.path.join(directory, name + '.png'), bbox_inches='tight', pad_inches=0)
            plt.close()
    except:
        print(f"fail to plot {file}")

plotting bounding box:   2%|▏         | 5/219 [00:13<13:34,  3.81s/it]